In [2]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 7.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=da174fa7ec89cc6677493ef9afbe48bfed744ec842b4bc6cc6c4f75cc35d6fb3
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=17cb4888e45aedf10f021a047e46e9d4233e93824a81b1c378f1a04b6eea9a94
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_filter_1', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_kernel_1', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_filter_2', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_kernel_2', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 22s]
val_accuracy: 0.8503333330154419

Best val_accuracy So Far: 0.9051666855812073
Total elapsed time: 00h 01m 55s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 80)        2080      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        96048     
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1536080   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 1,635,018
Trainable params: 1,635,018
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1750 - accuracy: 0.9343 - val_loss: 0.2677 - val_accuracy: 0.9077
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1384 - accuracy: 0.9482 - val_loss: 0.2868 - val_accuracy: 0.9103
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1089 - accuracy: 0.9586 - val_loss: 0.3332 - val_accuracy: 0.9060
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0835 - accuracy: 0.9679 - val_loss: 0.3103 - val_accuracy: 0.9080
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0694 - accuracy: 0.9746 - val_loss: 0.3618 - val_accuracy: 0.9120
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0510 - accuracy: 0.9806 - val_loss: 0.4155 - val_accuracy: 0.9093
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0430 - accuracy: 0.9844 - val_loss: 0.4280 - val_accuracy

In [16]:
model.evaluate(test_images,test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.4622 - accuracy: 0.9081


[0.46217623353004456, 0.9081000089645386]